In [1]:
# - *- coding: utf- 8*-
from LambdaNodeEmbedding import GetKerasNAP, KerasEval
import numpy as np
import tensorflow as tf
from keras import backend as K
from keras.models import Model
from keras.callbacks import History

#These are the edges between the nodes.
edgelist = np.array([
                    [0,1,1,1],
                    [0,0,0,1],
                    [0,1,0,0],
                    [0,0,0,0]
                    ], dtype='float32')

edgelist_ten = K.variable(value=edgelist, dtype='float32')

print("Input:\n", KerasEval(edgelist_ten))
print("Row_0:\n", KerasEval(edgelist_ten[0,:]))
print("Row_1:\n", KerasEval(edgelist_ten[1,:]))

new_x_x_x = tf.concat([edgelist_ten[0,:],edgelist_ten[1,:]], 0)
print('Concat_Test:\n', KerasEval(new_x_x_x))

def BuildZeroedConcatRow(edge_row, index):
    print('input:\n',KerasEval(edge_row))
    zero_calc = tf.zeros((index,), tf.float32)    
    keep_calc =  tf.ones(((edge_row.shape[0]-index),), tf.float32)
    concat_calc = tf.concat([zero_calc,keep_calc],0)
    result = tf.multiply(concat_calc, edge_row) 
    return result

def IndexOutOfBounds(edge_row, index):
    return tf.cond(tf.less_equal(index, edge_row.shape[0]), 
                   lambda: BuildZeroedConcatRow(edge_row, index),
                   lambda: tf.zeros(edge_row.shape, tf.float32))
    

def ZerofyUnderIndexInRow(edge_row, index):
    return tf.cond(tf.less(index, tf.Variable(1)), 
                   lambda: edge_row,
                   lambda: IndexOutOfBounds(edge_row, index))

def ZerofyUnderPrincipalDiagonal(edge_emb, result_emb, iterator):
    
    #Condition => if current iteration is lower as max iterations
    def cond(edge_emb, result_emb, i):
         return tf.less(i, iterator)

    #Loop Body Processing Unit => get all connection dependencies
    def body(edge_emb, result_emb, i):
        
        result_row = ZerofyUnderIndexInRow(edge_emb[i,:], i)
        
        #Store result in Tensorarray result_emb
        result_emb = result_row.write(i, [result_row])
        
        #Add +1 to Iterator and return process result!
        return [edge_emb, result_emb, tf.add(i, 1)]

    #return the loop setup
    return tf.while_loop(cond, body, [edge_emb, result_emb, 0])


Using TensorFlow backend.


Input:
 [[0. 1. 1. 1.]
 [0. 0. 0. 1.]
 [0. 1. 0. 0.]
 [0. 0. 0. 0.]]
Row_0:
 [0. 1. 1. 1.]
Row_1:
 [0. 0. 0. 1.]
Concat_Test:
 [0. 1. 1. 1. 0. 0. 0. 1.]


In [2]:
print(KerasEval(BuildZeroedConcatRow(new_x_x_x, 1)))
print('\n#######\n')
print(KerasEval(BuildZeroedConcatRow(new_x_x_x, 3)))
print('\n#######\n')
print(KerasEval(BuildZeroedConcatRow(new_x_x_x, 5)))
print('\n#######\n')
print(KerasEval(BuildZeroedConcatRow(new_x_x_x, 8)))

input:
 [0. 1. 1. 1. 0. 0. 0. 1.]
[0. 1. 1. 1. 0. 0. 0. 1.]

#######

input:
 [0. 1. 1. 1. 0. 0. 0. 1.]
[0. 0. 0. 1. 0. 0. 0. 1.]

#######

input:
 [0. 1. 1. 1. 0. 0. 0. 1.]
[0. 0. 0. 0. 0. 0. 0. 1.]

#######

input:
 [0. 1. 1. 1. 0. 0. 0. 1.]
[0. 0. 0. 0. 0. 0. 0. 0.]


In [3]:
print('Result_1:\n', KerasEval(IndexOutOfBounds(new_x_x_x, 2)))
print('\n#######\n')
print('Result_1:\n', KerasEval(IndexOutOfBounds(new_x_x_x, 3)))
print('\n#######\n')
print('Result_1:\n', KerasEval(IndexOutOfBounds(new_x_x_x, 7)))
print('\n#######\n')
print('Result_1:\n', KerasEval(IndexOutOfBounds(edgelist_ten[0,:], 4)))

input:
 [0. 1. 1. 1. 0. 0. 0. 1.]
Result_1:
 [0. 0. 1. 1. 0. 0. 0. 1.]

#######

input:
 [0. 1. 1. 1. 0. 0. 0. 1.]
Result_1:
 [0. 0. 0. 1. 0. 0. 0. 1.]

#######

input:
 [0. 1. 1. 1. 0. 0. 0. 1.]
Result_1:
 [0. 0. 0. 0. 0. 0. 0. 1.]

#######

input:
 [0. 1. 1. 1.]
Result_1:
 [0. 0. 0. 0.]


In [4]:
print(KerasEval(ZerofyUnderIndexInRow(new_x_x_x, 1)))

ValueError: Operation 'cond_4/cond/IsVariableInitialized' has been marked as not fetchable.

In [ ]:
#print(ZerofyUnderIndexInRow(new_x_x_x, -1))
#print('\n#######\n')
#print(ZerofyUnderIndexInRow(new_x_x_x, 1))
#print('\n#######\n')
#print(ZerofyUnderIndexInRow(new_x_x_x, 3))
#print('\n#######\n')
#print(ZerofyUnderIndexInRow(new_x_x_x, 9))